# Minimal 1-Node Investment Example

The example models a simple electricity system as a 1-node-model the following nodes (buses and components):

![](dispatch.png?=20)


The model will optimize the capacity of all supply technologies. 

## Python Imports

Imports of necessary python packages

In [1]:
import os
import pkg_resources as pkg
import pandas as pd 

from oemof.solph import EnergySystem, Model, Bus
from oemof.tools.economics import annuity as annuity
import oemof.tabular.tools.postprocessing as pp
import oemof.tabular.facades as fc

## Creating and Setting the Datapaths

Setting the datapath for raw-data and results. Data handling looks more complex than it is. You can easily adapt this to a simple `pd.read_excel(filepath,...)` in the next block if your file is located somewhere else. Otherwise we will use data from the oemof tabular repository. 

In addition a results directory will be created in `home/user/oemof-results/dispatch/output`. 

In [2]:
# datapath for input data from the oemof tabular pacakge
datapath = os.path.join(
    pkg.resource_filename("oemof.tabular", "").replace('src/oemof/tabular', ""), 
    "data/data.xls")

# results path for output
results_path = os.path.join(
    os.path.expanduser("~"), 
    "oemof-results", 
    "investment", 
    "output"
)

if not os.path.exists(results_path):
    os.makedirs(results_path)
datapath

'/home/admin/projects/oemof-tabular/data/data.xls'

Next we will read the required input data. The timeseries index will be used for the `EnergySystem` object below. 
All generator data etc. will also be loaded. 

In [3]:
timeseries = pd.read_excel(
    datapath, sheet_name="timeseries", index_col=[0], parse_dates=True
)
timeseries.index.freq = "1H"

generator = pd.read_excel(
    datapath, sheet_name="generator", index_col=0
)

storage = pd.read_excel(
    datapath, sheet_name="storage", index_col=0
)

carrier = pd.read_excel(
    datapath, sheet_name="carrier", index_col=0
)

technology = pd.read_excel(
    datapath, sheet_name="technology-data", index_col=0
)

load = pd.read_excel(
    datapath, sheet_name="load", index_col=0
)

In [4]:
load

,bus,amount,profile
name,,,
KH-load,KH-electricity,100000,KH-load-profile


## Creating the EnergySystem and its Nodes

We are starting by creating a `EnergySystem` object which will hold all information (nodes, etc.) of hour energy system that we will add below. This is just the standard way of using the `oemof.solph` library for your modelling.

In [ ]:
es = EnergySystem(timeindex=timeseries.index)

### Add Bus

Before we add any components we will create all bus objects for our model and add it to the energy system object.

In [ ]:
buses = {b: Bus(label=b) for b in set(generator.bus)}
es.add(*buses.values())

#### Bus Constraints 

With the set of all Buses $B$ all inputs $x^{flow}_{i(b),b}$ to a bus $b$ must equal all its outputs $x^{flow}_{b,o(b)}$

$$\sum_i x^{flow}_{i(b), b}(t) - \sum_o x^{flow}_{b, o(b)}(t) = 0 \qquad \forall t \in T, \forall b \in B$$

This equation will be build once the complete energy system is setup with its component. Every time a `Component` is created, the connected bus inputs/outputs will be updated. By this update every bus has all required information of its inputs and outputs available to construct the constraints. 


### Add Load


In [ ]:
for name, l in load.iterrows(): 
    es.add(
        fc.Load(
            label=name,       
            bus=buses[l.bus], # reference the bus in the buses dictionary
            amount=l.amount,  # amount column
            profile=timeseries[l.profile])
    )


#### Load Constraint 

For the set of all Load denoted with $l \in L$ the load $x_l$ at timestep t equals the exogenously defined  profile value $c^{profile}_l$ multiplied by the amount of this load $c^{amount}_l$

$$ x^{flow}_{l}(t) = c^{profile}_{l}(t) \cdot c^{amount}_{l} \qquad \forall t \in T, \forall l \in L$$

### Add Generators

In [ ]:
for name, g in generator.iterrows(): 
    if g.profile not in timeseries: 
        timeseries[g.profile] = 0
    es.add(
        fc.Generator(
            label=name,
            bus=buses[g.bus],
            carrier=g.carrier,
            tech=g.tech,
            marginal_cost=g.marginal_cost,
            capacity_cost=annuity(
                technology.at['capex', ("-").join([g.carrier, g.tech])], 
                technology.at['lifetime', ("-").join([g.carrier, g.tech])], 
                0.07),
            output_parameters={
               'fixed': not bool(g.dispatchable),
            },
            profile=timeseries[g.profile]
        )
    )
    

#### Dispatchable Generator Constraint

A `Generator` component can be used to model all types of dispatchble units in a energy system. This can include diesel generators oder coal fired power plants but also hot water boilers for heat. Every generator **must** be connected to an `Bus` object. 

This basic mathematical model for the component with the set of all dispatchable generators being $d \in D$ looks as follows:

$$x^{flow}_{d}(t) \leq x^{capacity}_{d} \qquad \forall t \in T,  \forall d \in D$$

Meaning, the production of the generator $x^{flow}$ must be less than its maximum capacity $c^{max}$ in every timestep. 

#### Volatile Generator Constraint 

Using the `Generator` component with `output_parameters={"fixed": True}` is very similar to the Dispatchable component. However, in this case the flow of the volatile components denoted with $v \in V$  will be fixed to a specific value.

$$ x^{flow}_{v}(t) = c^{profile}_{v}(t) \cdot x^{capacity}_{v} \qquad \forall t \in T, \forall v \in V$$

Alternatively you can use the `Volatile` component which automatically enforced the fixed behaviour. 

### Add Storage

In [ ]:
"""
es.add(
    fc.Storage(
        label="storage",
        bus=myamar,
        carrier="lithium",
        tech="battery",
        capacity=20,
        marginal_cost=0.0001,
        balanced=True,
        initial_storage_capacity=0.5,
        storage_capacity=100,
    )
)
es.add(fc.Excess(label="excess", bus=myamar))
"""

'\nes.add(\n    fc.Storage(\n        label="storage",\n        bus=myamar,\n        carrier="lithium",\n        tech="battery",\n        capacity=20,\n        marginal_cost=0.0001,\n        balanced=True,\n        initial_storage_capacity=0.5,\n        storage_capacity=100,\n    )\n)\nes.add(fc.Excess(label="excess", bus=myamar))\n'

#### Storage Constraints 

The mathematical representation of the storage for all storages $s \in S$ will include the flow into the storage, out of the storage and a storage level. The defaul efficiency for input/output is 1. Note that this is is included during charge and discharge. If you want to set the round trip efficiency you need to do for example: $\eta = \sqrt{\eta^{roundtrip}}$

Intertemporal energy balance of the storage:

$$ x^{level}_{s}(t) = \eta^{loss} x^{level}_{s}(t) + \eta x^{flow}_{s, in} - \eta x^{flow}_{s, out}(t) \qquad \forall t \in T,  \forall s \in S$$ 

Bounds of the storage level variable $x^{level}_s(t)$:

$$ x^{level}_s(t) \leq c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$


$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$ 

Of course, in addition the inflow/outflow of the storage also needs to be within the limit of the minimum and maximum power. 

$$ -c_s^{capacity} \leq x^{flow}_s(t) \leq c_s^{capacity} \qquad \forall t \in T, \forall s \in S$$ 


### Objective Function 

The objective function is created from all instantiated objects. It will use all operating costs (i.e. `marginal_cost` argument) and if set all investment costs (i.e. `capacity_cost` argument)

$$ \text{min:} \sum_g \sum_t \overbrace{c^{marginal\_cost}_g \cdot x^{flow}_{g}(t)}^{\text{operating cost}} \\ 
\sum_g \sum_t \overbrace{c^{capacity\_cost}_g \cdot x^{capacity}_{g}(t)}^{\text{investment cost}} $$

## Creating the Mathematical Model

In [ ]:
# create model based on energy system and its components
m = Model(es)

# inspect objective function
# m.objective.pprint()

## Solving the Model and Writing Results

In [ ]:

#  solve the model using cbc solver
m.solve("cbc")

# write results back to the model object
m.results = m.results()

# writing results with the standard oemof-tabular output formatt
pp.write_results(m, results_path)

print("Optimization done. Results are in {}.".format(results_path))


> /home/admin/projects/oemof-tabular/src/oemof/tabular/tools/postprocessing.py(123)supply_results()
-> selection = selection.loc[
(Pdb) selection
from                          KH-coal-st   KH-hydro-ror    KH-solar-pv  \
to                        KH-electricity KH-electricity KH-electricity   
type                                flow           flow           flow   
timeindex                                                                
2030-01-01 00:00:00+00:00       5.268469       3.123848       0.000000   
2030-01-01 01:00:00+00:00       4.781602       3.122354       0.000000   
2030-01-01 02:00:00+00:00       4.361871       3.120854       0.000000   
2030-01-01 03:00:00+00:00       4.075530       3.119346       0.000000   
2030-01-01 04:00:00+00:00       3.713992       3.117831       0.000000   
2030-01-01 05:00:00+00:00       3.106873       3.116309       0.000000   
2030-01-01 06:00:00+00:00       3.110577       3.114780       0.000000   
2030-01-01 07:00:00+00:00       3.249792

(Pdb) views.node_output_by_type(results, node_type=es.typemap[t])
(Pdb) views.node_output_by_type(results, node_type=es.typemap[t])
(Pdb) t
'reservoir'
(Pdb) views.node_output_by_type(results, node_type=es.typemap['generator'])
from                          KH-coal-st   KH-hydro-ror    KH-solar-pv  \
to                        KH-electricity KH-electricity KH-electricity   
type                                flow           flow           flow   
timeindex                                                                
2030-01-01 00:00:00+00:00       5.268469       3.123848       0.000000   
2030-01-01 01:00:00+00:00       4.781602       3.122354       0.000000   
2030-01-01 02:00:00+00:00       4.361871       3.120854       0.000000   
2030-01-01 03:00:00+00:00       4.075530       3.119346       0.000000   
2030-01-01 04:00:00+00:00       3.713992       3.117831       0.000000   
2030-01-01 05:00:00+00:00       3.106873       3.116309       0.000000   
2030-01-01 06:00:00+00:00       

## Plotting the Results

In [ ]:
import os
from plotly import offline, plotly
from oemof.tabular.tools.plots import hourly_plot
offline.init_notebook_mode()

name = "investment"

# results path for output
results_path = os.path.join(os.path.expanduser("~"), "oemof-results", name, "output")

offline.plot(
    hourly_plot(
        name,
        "KH-electricity",
        os.path.join(os.path.expanduser("~"), "oemof-results"),
        plot_filling_levels=False,
    ),
    filename=os.path.join(results_path, "hourly-plot.html"),
)